In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from skimage.color import rgb2gray
from skimage.transform import rescale
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import cv2

In [2]:
img = list()
gray = list()
for i in range(441):
    img.append(cv2.imread(f'{i}.jpg'))
    gray.append(cv2.cvtColor(img[i], cv2.COLOR_BGR2GRAY))


img = np.array(img)
gray = np.array(gray)

In [21]:
orb = cv2.ORB_create(nfeatures=10)
outer = list()
for i in range(441):
    inner = list()
    for j in range(0,441,10):
        img1 = gray[i]
        img2 = gray[j]


        kp1, des1 = orb.detectAndCompute(img1, None)
        kp2, des2 = orb.detectAndCompute(img2, None)

        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)
        if(len(matches) != 0):
            inner.append(matches[0].distance)
        else:
            inner.append(999)
        
        inner.append(len(matches))
    
    print(i)
    outer.append(inner)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
X = np.array(outer)

In [23]:
target = pd.read_csv("coordinates.csv")
Y = np.array(target.values.tolist())

In [36]:
kf = KFold(n_splits=5, shuffle= True)

mse = list()
train_mse = list()

for train_index, test_index in kf.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model = Sequential()
    model.add(Dense(100, activation='relu',input_shape=X[0].shape))
    model.add(Dropout(0.01))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2))
    
    model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])
    model.fit(x_train, y_train, epochs=75, batch_size=3)
    results = model.predict(x_test)
    
    train_mse.append(mean_squared_error(y_train,model.predict(x_train)))
    mse.append(mean_squared_error(y_test,results))

Epoch 1/75
118/118 [==============================] - 0s 937us/step - loss: 7331.5674 - mean_squared_error: 7331.5674
Epoch 2/75
118/118 [==============================] - 0s 846us/step - loss: 6800.7114 - mean_squared_error: 6800.7114
Epoch 3/75
118/118 [==============================] - 0s 768us/step - loss: 5310.5830 - mean_squared_error: 5310.5830
Epoch 4/75
118/118 [==============================] - 0s 766us/step - loss: 5406.4990 - mean_squared_error: 5406.4990
Epoch 5/75
118/118 [==============================] - 0s 771us/step - loss: 6408.7925 - mean_squared_error: 6408.7925
Epoch 6/75
118/118 [==============================] - 0s 803us/step - loss: 4984.7217 - mean_squared_error: 4984.7217
Epoch 7/75
118/118 [==============================] - 0s 845us/step - loss: 4718.5308 - mean_squared_error: 4718.5308
Epoch 8/75
118/118 [==============================] - 0s 784us/step - loss: 4247.5679 - mean_squared_error: 4247.5674
Epoch 9/75
118/118 [==============================] - 0s

118/118 [==============================] - 0s 811us/step - loss: 2609.1155 - mean_squared_error: 2609.1155
Epoch 70/75
118/118 [==============================] - 0s 752us/step - loss: 2358.9336 - mean_squared_error: 2358.9336
Epoch 71/75
118/118 [==============================] - 0s 735us/step - loss: 2352.1758 - mean_squared_error: 2352.1758
Epoch 72/75
118/118 [==============================] - 0s 740us/step - loss: 2199.6697 - mean_squared_error: 2199.6697
Epoch 73/75
118/118 [==============================] - 0s 729us/step - loss: 2294.0459 - mean_squared_error: 2294.0459
Epoch 74/75
118/118 [==============================] - 0s 745us/step - loss: 2274.4836 - mean_squared_error: 2274.4836
Epoch 75/75
118/118 [==============================] - 0s 749us/step - loss: 2281.7424 - mean_squared_error: 2281.7424
Epoch 1/75
118/118 [==============================] - 0s 797us/step - loss: 6737.4556 - mean_squared_error: 6737.4556
Epoch 2/75
118/118 [==============================] - 0s 819u

118/118 [==============================] - 0s 803us/step - loss: 2205.0034 - mean_squared_error: 2205.0037
Epoch 63/75
118/118 [==============================] - 0s 750us/step - loss: 2228.2493 - mean_squared_error: 2228.2493
Epoch 64/75
118/118 [==============================] - 0s 746us/step - loss: 2203.5271 - mean_squared_error: 2203.5271
Epoch 65/75
118/118 [==============================] - 0s 745us/step - loss: 2361.4314 - mean_squared_error: 2361.4314
Epoch 66/75
118/118 [==============================] - 0s 741us/step - loss: 2249.2412 - mean_squared_error: 2249.2412
Epoch 67/75
118/118 [==============================] - 0s 765us/step - loss: 2212.2942 - mean_squared_error: 2212.2942
Epoch 68/75
118/118 [==============================] - 0s 738us/step - loss: 2264.4919 - mean_squared_error: 2264.4919
Epoch 69/75
118/118 [==============================] - 0s 765us/step - loss: 2335.5325 - mean_squared_error: 2335.5325
Epoch 70/75
118/118 [==============================] - 0s 73

118/118 [==============================] - 0s 771us/step - loss: 1968.8610 - mean_squared_error: 1968.8610
Epoch 57/75
118/118 [==============================] - 0s 779us/step - loss: 2102.9338 - mean_squared_error: 2102.9338
Epoch 58/75
118/118 [==============================] - 0s 778us/step - loss: 2011.4886 - mean_squared_error: 2011.4886
Epoch 59/75
118/118 [==============================] - 0s 775us/step - loss: 2142.5911 - mean_squared_error: 2142.5911
Epoch 60/75
118/118 [==============================] - 0s 745us/step - loss: 2136.3647 - mean_squared_error: 2136.3647
Epoch 61/75
118/118 [==============================] - 0s 782us/step - loss: 1941.9053 - mean_squared_error: 1941.9053
Epoch 62/75
118/118 [==============================] - 0s 763us/step - loss: 2016.0652 - mean_squared_error: 2016.0652
Epoch 63/75
118/118 [==============================] - 0s 763us/step - loss: 1976.2452 - mean_squared_error: 1976.2452
Epoch 64/75
118/118 [==============================] - 0s 78

118/118 [==============================] - 0s 820us/step - loss: 2666.8669 - mean_squared_error: 2666.8669
Epoch 51/75
118/118 [==============================] - 0s 781us/step - loss: 2565.0884 - mean_squared_error: 2565.0884
Epoch 52/75
118/118 [==============================] - 0s 765us/step - loss: 2600.8047 - mean_squared_error: 2600.8047
Epoch 53/75
118/118 [==============================] - 0s 746us/step - loss: 2546.1282 - mean_squared_error: 2546.1282
Epoch 54/75
118/118 [==============================] - 0s 785us/step - loss: 2536.2234 - mean_squared_error: 2536.2234
Epoch 55/75
118/118 [==============================] - 0s 759us/step - loss: 2865.2964 - mean_squared_error: 2865.2964
Epoch 56/75
118/118 [==============================] - 0s 746us/step - loss: 2591.9771 - mean_squared_error: 2591.9771
Epoch 57/75
118/118 [==============================] - 0s 779us/step - loss: 2488.0813 - mean_squared_error: 2488.0813
Epoch 58/75
118/118 [==============================] - 0s 77

118/118 [==============================] - 0s 803us/step - loss: 2688.0977 - mean_squared_error: 2688.0977
Epoch 45/75
118/118 [==============================] - 0s 760us/step - loss: 2766.0627 - mean_squared_error: 2766.0627
Epoch 46/75
118/118 [==============================] - 0s 737us/step - loss: 2659.7305 - mean_squared_error: 2659.7305
Epoch 47/75
118/118 [==============================] - 0s 773us/step - loss: 2681.8914 - mean_squared_error: 2681.8914
Epoch 48/75
118/118 [==============================] - 0s 754us/step - loss: 2617.1250 - mean_squared_error: 2617.1250
Epoch 49/75
118/118 [==============================] - 0s 754us/step - loss: 2601.2583 - mean_squared_error: 2601.2583
Epoch 50/75
118/118 [==============================] - 0s 752us/step - loss: 2637.4397 - mean_squared_error: 2637.4397
Epoch 51/75
118/118 [==============================] - 0s 750us/step - loss: 2596.2969 - mean_squared_error: 2596.2969
Epoch 52/75
118/118 [==============================] - 0s 77

In [37]:
sum(mse)/5

5156.044302569295

In [38]:
sum(train_mse)/5

2102.09253925364